In [137]:
from dwave.system import EmbeddingComposite, DWaveSampler
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
import pandas as pd

In [142]:
# load the qpu and the auto embedding solver
qpu = DWaveSampler(region='eu-central-1')
print(qpu.solver.name)
sampler = EmbeddingComposite(qpu)
MAX_QPU_TIME = qpu.properties['problem_run_duration_range'][-1]   # in microseconds for Advantage_5.4

Advantage_system5.4


# Problem setup

We want to find out the statistical behaviour of dwave's qpu.
To do this, we first need to define the problem.

We consider the following two qubit Ising problem:
$$J_{0,1} = -1$$
Such a problem has 2 possible solutions:
$\ket{00}$ and $\ket{11}$.

We need to find out the distribution of the qubits after a certain number of steps.
Statistically we expect them to be equally distributed regardless of :
- The qubits on which they are operated
- The annealing time
- The number of samples per trial
- The mode of the annealing schedule ( simple forward or reverse+forward annealing ) 

# Simple examples

## Forward annealing example

In [88]:
J = {(0, 1): -1}

In [119]:
sampleset_forward = sampler.sample_ising({},J, num_reads=450, annealing_time=2000, auto_scale=False).to_pandas_dataframe()

In [90]:
sampleset_forward['state'] = sampleset_forward.apply(lambda row: f"{int((row[0]+1)/2)} {int((row[1]+1)/2)}", axis=1)
px.bar(sampleset_forward, x='state',y='num_occurrences', title='Frequency distribution', text='num_occurrences')

## Reverse annealing example

In [91]:
initial = {0:-1, 1:-1}  # Set the initial state
reverse_schedule = [ [0.0,1.0], [50,0.5], [100,1.0] ] # Set the annealing schedule, first element is % , the second is ratio of mixing hamiltonian and problem hamiltonian
reverse_anneal_params = dict(anneal_schedule=reverse_schedule,
                             initial_state=initial,
                             reinitialize_state=True)  # Reinitialize to the specified initial state for every anneal-readout cycle. Each anneal begins from the state given in the initial_state parameter.
# See https://docs.dwavesys.com/docs/latest/c_solver_parameters.html#reinitialize-state


In [92]:
sampleset_reverse = sampler.sample_ising({}, J, num_reads=1000, **reverse_anneal_params).to_pandas_dataframe()

In [93]:
sampleset_reverse['state'] = sampleset_reverse.apply(lambda row: f"{int((row[0]+1)/2)} {int((row[1]+1)/2)}", axis=1)
px.bar(sampleset_reverse.groupby('state').count().reset_index(), x='state', y='num_occurrences', title='Frequency distribution', text='num_occurrences')

In [94]:
sampleset_reverse

,0,1,chain_break_fraction,energy,num_occurrences,state
0,-1,-1,0.0,-1.0,1,0 0
1,1,1,0.0,-1.0,1,1 1
2,-1,-1,0.0,-1.0,1,0 0
3,1,1,0.0,-1.0,1,1 1
4,-1,-1,0.0,-1.0,1,0 0
...,...,...,...,...,...,...
995,-1,-1,0.0,-1.0,1,0 0
996,-1,-1,0.0,-1.0,1,0 0
997,-1,-1,0.0,-1.0,1,0 0
998,1,1,0.0,-1.0,1,1 1


# Sweeps

## Forward annealing sweep across annealing times

Fix the actual qubit and number of reads per trial and sweep the annealing times

In [101]:
q0 = qpu.nodelist[1]
q1 =  next(iter(qpu.adjacency[q0]))
q0, q1  # The two chosen qubits

(31, 32)

In [102]:
qpu.adjacency[q0]

{30, 32, 46, 3120, 3135, 3150, 3165}

In [161]:
def submit_ising_problem(qpu, q0, q1, num_reads_per_trial, annealing_time):
    """Submits an Ising problem to the QPU, handling potential long runtimes.

    Args:
        qpu: The QPU object.
        q0: The first qubit integer on the qpu.
        q1: The second qubit.
        num_reads_per_trial: The number of reads per trial.
        annealing_time: The annealing time.

    Returns:
        A Pandas DataFrame containing the sampleset.
    """

    ising_J_couplings =  {(q0, q1): -1}
    ising_H_couplings =  {}

    qpu_estimate = qpu.solver.estimate_qpu_access_time(
        num_qubits=2,
        num_reads=num_reads_per_trial,
        annealing_time=annealing_time
    ) * 1.05

    if qpu_estimate > MAX_QPU_TIME:
            # then we break the num_reads into smaller chunks
            # A gothca here is that we submit multiple jobs which may or may not run sequentially
            # even if they do , there would be a few micro secs delay between the jobs,
            # and we wont be so sure that just merging the results is a good idea
        overtime = qpu_estimate / MAX_QPU_TIME
        chunks = [int(num_reads_per_trial / overtime)] * int(overtime)
        if sum(chunks) < num_reads_per_trial:
            chunks.append(num_reads_per_trial - sum(chunks))

        sampleset = pd.DataFrame()
        for chunk_id, chunk_size in enumerate(chunks):
            temp_sampleset = qpu.sample_ising(
                ising_H_couplings,
                ising_J_couplings,
                num_reads=chunk_size,
                annealing_time=annealing_time
            ).to_pandas_dataframe()
            temp_sampleset['chunk_id'] = chunk_id
            temp_sampleset['chunk_size'] = chunk_size
            sampleset = pd.concat([sampleset, temp_sampleset], ignore_index=True)
    else:
        sampleset = qpu.sample_ising(
            ising_H_couplings,
            ising_J_couplings,
            num_reads=num_reads_per_trial,
            annealing_time=annealing_time
        ).to_pandas_dataframe()
        sampleset['chunk_id'] = 0
        sampleset['chunk_size'] = num_reads_per_trial

    return sampleset

In [162]:
submit_ising_problem(qpu, q0, q1, 1000, 2000)

,31,32,energy,num_occurrences,chunk_id,chunk_size
0,-1,-1,-1.0,196,0,460
1,1,1,-1.0,264,0,460
2,1,1,-1.0,235,1,460
3,-1,-1,-1.0,225,1,460
4,1,1,-1.0,47,2,80
5,-1,-1,-1.0,33,2,80


In [158]:
num_reads_per_trial = 2000
annealing_time = 2000
qpu_estimate = qpu.solver.estimate_qpu_access_time(num_qubits=2, num_reads=num_reads_per_trial,annealing_time=annealing_time) *1.05
if  qpu_estimate > MAX_QPU_TIME:
    # then we break the num_reads into smaller chunks
    # A gothca here is that we submit multiple jobs which may or may not run sequentially
    # even if they do , there would be a few micro secs delay between the jobs,
    # and we wont be so sure that just merging the results is a good idea
    overtime = (qpu_estimate / MAX_QPU_TIME)
    # split the number into a list of trials which sum to the original number
    chunks = [int(num_reads_per_trial / overtime)] * int(overtime)
    if sum(chunks) < num_reads_per_trial:
        chunks.append( num_reads_per_trial - sum(chunks) )

    # we now submit the jobs for each chunk and then merge the results
    sampleset = qpu.sample_ising({},{(q0,q1):-1}, num_reads=chunks[0], annealing_time = annealing_time).to_pandas_dataframe()
    sampleset['chunk_id'] = 0
    sampleset['chunk_size'] = chunks[0]
    for id, chunk in enumerate(chunks[1:]):
        temp_sampleset = qpu.sample_ising({},{(q0,q1):-1}, num_reads=chunk, annealing_time = annealing_time).to_pandas_dataframe()
        temp_sampleset['chunk_size'] = chunk
        temp_sampleset['chunk_id'] = id
        sampleset = pd.concat([sampleset, temp_sampleset], ignore_index=True)


else:
    sampleset = qpu.sample_ising({},{(q0,q1):-1}, num_reads=num_reads_per_trial, annealing_time = annealing_time).to_pandas_dataframe()
    sampleset['chunk_id'] = 0
    sampleset['chunk_size'] = num_reads_per_trial

In [147]:
chunks

[461, 461, 461, 461, 156]

In [159]:
sampleset

,31,32,energy,num_occurrences,chunk_id,chunk_size
0,-1,-1,-1.0,225,0,461
1,1,1,-1.0,236,0,461
2,1,1,-1.0,251,0,461
3,-1,-1,-1.0,210,0,461
4,-1,-1,-1.0,208,1,461
5,1,1,-1.0,253,1,461
6,-1,-1,-1.0,225,2,461
7,1,1,-1.0,236,2,461
8,-1,-1,-1.0,85,3,156
9,1,1,-1.0,71,3,156


In [149]:
total_no_sample = 1_000_000
num_trials = 1000
num_reads_per_trial = int(total_no_sample / num_trials)

for annealing_time in np.linspace( *qpu.properties["annealing_time_range"], num_trials):
    annealing_time = round(annealing_time, 2)
    sampleset = qpu.sample_ising({},{(q0,q1):-1}, num_reads=num_reads_per_trial, annealing_time = annealing_time)
    sampleset.to_pandas_dataframe().to_csv(f'bell_data/{annealing_time}-{q0}-{q1}-{num_trials}.csv')

OSError: Cannot save file into a non-existent directory: 'bell_data'